In [1]:
from bs4 import BeautifulSoup
#from bs4 import soupStrainer
from urllib.request import urlopen
import pandas as pd
import re
import requests

In [2]:
from string import ascii_uppercase as alc

In [3]:
path = f"https://charleyproject.org/case-updates/resolved?letter=A"
r = requests.get(path)
soup = BeautifulSoup(r.text, 'html.parser')
linksToCases = soup.find_all('a', href=re.compile('^https://charleyproject.org/case/'))
final_links = []
for caselink in linksToCases:
    final_links.append(caselink.get('href'))

In [4]:
from string import ascii_uppercase as alc
from tqdm import tqdm
final_links = []
for i in tqdm(alc):
    path = f"https://charleyproject.org/case-updates/resolved?letter={i}"
    r = requests.get(path)
    soup = BeautifulSoup(r.text, 'html.parser')
    linksToCases = soup.find_all('a', href=re.compile('^https://charleyproject.org/case/'))
    
    for caselink in linksToCases:
        final_links.append(caselink.get('href'))

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:27<00:00,  1.06s/it]


In [5]:
r = requests.get(path)
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all('div', attrs={'class':'cases'})

In [6]:
link = []
for a_tag in soup.find_all('a'):
    #if link[link['Links:'].str.contains("https://charleyproject.org/case/")]:
    link.append(a_tag.get('href'))

In [7]:
#if contains 'https://charleyproject.org/case/ return to new df.
link = pd.DataFrame(final_links)
link = link.rename(columns={0: 'Links:'})
link = link.dropna()
link = link[link['Link:'].str.contains("https://charleyproject.org/case/")]
link = link.reset_index(drop=True)

KeyError: 'Link:'

In [9]:
resolved_df = pd.DataFrame()

In [ ]:
#details = []
for i in tqdm(range(0, (link.size-1))):
    link_str = link.at[i, 'Links:']
    r2 = requests.get(link_str)
    soup2 = BeautifulSoup(r2.text, 'html.parser')

    person = {"Link:": [link_str],
            "Name": [soup2.find('h1').get_text()],}
    
    element = soup2.find_all('p')
    
    #details = {"Links:": [link_str],
            #"Details": 
    #details.append(element[1])
   
    #print(details)
        
    #i = 0

    for strong_tag in soup2.find_all('strong'):
        k = strong_tag.text.strip()
        v = strong_tag.next_sibling.strip()
        person[k] = [v]

        #i+=1
        
    person["Details of Disappearance"] = [element[1].string]


    persondf = pd.DataFrame(person)

    if not resolved_df.empty:
        resolved_df = pd.merge(resolved_df, persondf, how='outer')
    else:
        resolved_df = persondf


resolved_df

 43%|█████████████████████████████████▉                                             | 811/1886 [10:58<14:56,  1.20it/s]

In [ ]:
resolved_df['Missing From'] = resolved_df['Missing From'].str.replace(' 								', ' ')

In [ ]:
resolved_df['Details of Disappearance'] = resolved_df['Details of Disappearance'].str.replace('[', '')

In [ ]:
resolved_df['Details of Disappearance']

In [ ]:
resolved_df.to_excel("all_together_now.xlsx")